In [21]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras import regularizers

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
f = open('/content/drive/My Drive/Colab Notebooks/News_Category_Dataset.json','r')
df = pd.read_json(f, lines=True, orient='records')

In [0]:
df.drop(['date','link'],axis=1,inplace=True)

In [26]:
df.columns

Index(['authors', 'category', 'headline', 'short_description'], dtype='object')

In [0]:
df_down = df[df.category=='POLITICS']
df_down = df_down[:3500]
df  = df[df.category != 'POLITICS']

In [0]:
df = pd.concat([df,df_down])

In [0]:
df_down = df[df.category=='ENTERTAINMENT']
df_down = df_down[:3500]
df  = df[df.category != 'ENTERTAINMENT']
df = pd.concat([df,df_down])

In [30]:
df = shuffle(df)
df.shape

(84993, 4)

In [0]:
X = df.drop('category',axis=1)
Y = df.category

In [32]:
print(X.shape,Y.shape)

(84993, 3) (84993,)


In [33]:
X['data'] = X.authors+" "+X.headline
X = X['data']
print(X.shape)

(84993,)


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = to_categorical(Y,num_classes=len(le.classes_))

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=0,test_size=0.2)

In [37]:
X_train.shape

(67994,)

In [0]:
words=5000
max_len = 150
tok = Tokenizer(num_words=words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
data = sequence.pad_sequences(sequences,maxlen=max_len)

In [39]:
model = Sequential()
model.add(Embedding(words, 100, input_length=max_len))
# model.add(Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_len,trainable=False))
model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
# model.add(Dense(31,activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(31, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 100)          500000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 31)                3131      
Total params: 583,531
Trainable params: 583,531
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(data, Y_train, validation_split=0.2, epochs=8, batch_size=128)

Train on 54395 samples, validate on 13599 samples
Epoch 1/8
54395/54395 [==============================] - 204s 4ms/step - loss: 3.1501 - acc: 0.1030 - val_loss: 2.5834 - val_acc: 0.1905
Epoch 2/8
54395/54395 [==============================] - 201s 4ms/step - loss: 2.0890 - acc: 0.4007 - val_loss: 1.6130 - val_acc: 0.5554
Epoch 3/8
54395/54395 [==============================] - 201s 4ms/step - loss: 1.4890 - acc: 0.5868 - val_loss: 1.3927 - val_acc: 0.6098
Epoch 4/8
54395/54395 [==============================] - 201s 4ms/step - loss: 1.2741 - acc: 0.6428 - val_loss: 1.3189 - val_acc: 0.6284
Epoch 5/8
54395/54395 [==============================] - 200s 4ms/step - loss: 1.1549 - acc: 0.6693 - val_loss: 1.2848 - val_acc: 0.6420
Epoch 6/8
54395/54395 [==============================] - 201s 4ms/step - loss: 1.0750 - acc: 0.6886 - val_loss: 1.2783 - val_acc: 0.6434
Epoch 7/8
54395/54395 [==============================] - 201s 4ms/step - loss: 1.0152 - acc: 0.7018 - val_loss: 1.2751 - val_acc

In [0]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [42]:
accr = model.evaluate(test_sequences_matrix,Y_test)

16999/16999 [==============================] - 84s 5ms/step


In [43]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.325
  Accuracy: 0.640
